In [11]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
load_dotenv()
apikey = os.getenv("groq_api_key")
llm = ChatGroq(
    temperature=0.6,
    groq_api_key=apikey,
    model="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was..")
print(response)

content='The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 44, 'total_tokens': 81, 'completion_time': 0.148, 'prompt_time': 0.008261395, 'queue_time': 0.005526794, 'total_time': 0.156261395}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b6828be2c9', 'finish_reason': 'stop', 'logprobs': None} id='run-65d94a79-5a1a-4401-b082-313e9aa42c8f-0' usage_metadata={'input_tokens': 44, 'output_tokens': 37, 'total_tokens': 81}


In [7]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.amazon.jobs/en/jobs/2672877/machine-learning-engineer-sponsored-display")
page_data = loader.load().pop().page_content
print(page_data)

Machine Learning Engineer, Sponsored Display - Job ID: 2672877 | Amazon.jobs
Skip to main contentHomeYour job applicationAmazon culture & benefitsDiversity at AmazonLocationsTeamsJob categoriesResourcesInterview tipsDisability accommodationsAbout AmazonFAQ×Machine Learning Engineer, Sponsored DisplayJob ID: 2672877 | Amazon Development Centre (Scotland) LimitedApply nowView transcriptDESCRIPTIONWant to build machine learning systems that impact hundreds of millions of customers worldwide?We are looking for outstanding Machine Learning Engineers to join the Audience Building teams within Amazon Ads. Audience Building enables advertisers to define the customers they want to engage based on shopping interactions, digital content consumption, and other events on Amazon via machine learning models and rule-based criteria. As a member of our Agile team you will have the opportunity to drive a key product in the Amazon Advertising targeting infrastructure.Your team will own the business, tech

from langa

In [10]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the careers's page of a website.
    your job is to extract the job postings and return then in JSON format containing the following
    keys :`role`,`experience`,`skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={"page_data":page_data})
print(res.content)

```
{
  "role": "Machine Learning Engineer, Sponsored Display",
  "experience": "Experience contributing to the architecture and design of new and current systems, experience in professional software development, experience programming with at least one modern language",
  "skills": ["Java", "C++", "C#", "Machine Learning", "Deep Learning", "Batch and real-time data processing", "Model containerization", "CI/CD pipelines", "API development"],
  "description": "Want to build machine learning systems that impact hundreds of millions of customers worldwide? We are looking for outstanding Machine Learning Engineers to join the Audience Building teams within Amazon Ads."
}
```


In [22]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Machine Learning Engineer, Sponsored Display',
 'experience': ['Experience contributing to the architecture and design (architecture, design patterns, reliability and scaling) of new and current systems',
  'Experience in professional, non-internship software development',
  'Experience programming with at least one modern language such as Java, C++, or C# including object-oriented design',
  'Experience in developing and deploying large scale machine learning or deep learning models and/or systems into production, including batch and real-time data processing, model containerization, CI/CD pipelines, API development, model training and productionizing ML models'],
 'skills': ['Architecture and design',
  'Java',
  'C++',
  'C#',
  'Object-oriented design',
  'Machine learning',
  'Deep learning',
  'Batch and real-time data processing',
  'Model containerization',
  'CI/CD pipelines',
  'API development'],
 'description': 'Want to build machine learning systems that impact h

In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("app/resource/techstackcsv.csv")

In [5]:
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular, Express, MySQL",https://example.com/angular-portfolio
2,"Vue, Flask, PostgreSQL",https://example.com/vue-portfolio
3,"Svelte, Node.js, SQLite",https://example.com/svelte-portfolio
4,"React, Django, MongoDB",https://example.com/react-django
5,"Next.js, Express, MongoDB",https://example.com/nextjs-portfolio
6,"Nuxt.js, FastAPI, MySQL",https://example.com/nuxtjs-portfolio
7,"HTML, CSS, Vanilla JS",https://example.com/html-css
8,"Flutter, Django, SQLite",https://example.com/flutter-django11
9,"Vue, Laravel, MySQL",https://example.com/vue-laravel12


In [27]:
import chromadb
import uuid
client = chromadb.PersistentClient("vectorstore")
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _,row in df.iterrows():
        collection.add(
            documents=row["Techstack"],
            metadatas={"links":row["Links"]},
            ids=[str(uuid.uuid4())]
        )

In [28]:
links = collection.query(query_texts=json_res["skills"],n_results=1).get('metadatas',[])
links

[[{'links': 'https://example.com/html-css'}],
 [{'links': 'https://example.com/sveltekit13'}],
 [{'links': 'https://example.com/html-css'}],
 [{'links': 'https://example.com/angular-dotnet29'}],
 [{'links': 'https://example.com/html-css'}],
 [{'links': 'https://example.com/html-css'}],
 [{'links': 'https://example.com/vue-dotnet17'}],
 [{'links': 'https://example.com/sveltekit13'}],
 [{'links': 'https://example.com/ember-flask27'}],
 [{'links': 'https://example.com/nuxtjs-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'}]]

In [16]:
json_res

[{'role': 'Retail Associate, SEAS - Houston',
  'experience': 'None specified',
  'skills': ['Positive attitude',
   'Knowledge of footwear, apparel and accessories',
   'Ability to work in a team environment',
   'Physical requirements include the ability to bend, squat, reach, climb a ladder and stand for extended periods of time'],
  'description': 'Converse is looking for a Retail Associate to join their team in Houston. As a Retail Associate, you will be responsible for creating a fun and knowledgeable environment for customers, supporting the customer sales journey, and acting in accordance with store policies and procedures.'}]

In [18]:
job = json_res[0]
job["skills"]

['Positive attitude',
 'Knowledge of footwear, apparel and accessories',
 'Ability to work in a team environment',
 'Physical requirements include the ability to bend, squat, reach, climb a ladder and stand for extended periods of time']

In [30]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION
    {job_description}
    ### INSTRUCTION:
    You are Sai, a business development executive at TCS. TCS is an AI & Software consultive company
    dedicated and provide the seamless integration of business process through automated tools.
    Over our experience, we have empowered numerous enterprises with tailored solutions, foestering the 
    process optimization, cost reduction, and heightened overall efficiency.
    Your job is to write a cold email to the client regarding the job mentioned above describing what we do
    and how we help in fulfilling their needs.
    Also add the most revelevant ones from the following links to showcase TCS portfolio:{link_list}
    
    Remember you are Sai, BDE at TCS.
    Do not provide any preamble
    ### EMAIL (NO PREAMBLE):
    """
)

chain_extract = prompt_email | llm
res = chain_extract.invoke(input={"job_description":str(json_res),"link_list":links})
print(res.content)

Subject: Expert Machine Learning Solutions for Amazon Ads

Dear Hiring Manager,

I came across the Machine Learning Engineer, Sponsored Display role at Amazon Ads and was impressed by the impact your team is making in developing machine learning systems that influence hundreds of millions of customers worldwide. As a Business Development Executive at TCS, I'd like to introduce you to our AI and software consulting services that can support your mission.

At TCS, we empower enterprises by seamlessly integrating business processes through automated tools. Our expertise in machine learning, deep learning, and software development can help you build and deploy large-scale machine learning models and systems into production. Our services include architecture and design, batch and real-time data processing, model containerization, CI/CD pipelines, and API development.

Our team has a proven track record of delivering high-quality solutions. For instance, we've developed a robust e-commerce p